# Mask detection model with CNN

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os, split_folders
import cv2
import numpy as np

Using TensorFlow backend.


## Part 1 - Data Preprocessing

### Preprocessing the training set

In [2]:
# Generate training set and test set
data_path = 'dataset'
split_folders.ratio('dataset', output="datasets", seed=1159, ratio=(.8, .2)) # default values

Copying files: 1376 files [02:11, 10.44 files/s]


In [3]:
img_size =(100,100)
# apply data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip = False)
training_set = train_datagen.flow_from_directory('datasets/train',
                                                 target_size = img_size,
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1100 images belonging to 2 classes.


### Preprocessing the test set

The image dataset was prepared by [Prajna Bhandary](https://github.com/prajnasb/observations/tree/master/experiements/data)

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('datasets/val',
                                            target_size = img_size,
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 276 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=200, kernel_size=3, activation='relu', input_shape=[100, 100, 3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=100, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=2, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the training set and evaluating it on the test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
35/35 [==============================] - 168s 5s/step - loss: 0.6422 - accuracy: 0.6445 - val_loss: 0.3435 - val_accuracy: 0.9130
Epoch 2/20
35/35 [==============================] - 171s 5s/step - loss: 0.3698 - accuracy: 0.8591 - val_loss: 0.2405 - val_accuracy: 0.9058
Epoch 3/20
35/35 [==============================] - 115s 3s/step - loss: 0.3084 - accuracy: 0.8945 - val_loss: 0.2701 - val_accuracy: 0.9022
Epoch 4/20
35/35 [==============================] - 148s 4s/step - loss: 0.2809 - accuracy: 0.8973 - val_loss: 0.1627 - val_accuracy: 0.9565
Epoch 5/20
35/35 [==============================] - 115s 3s/step - loss: 0.2240 - accuracy: 0.9182 - val_loss: 0.1766 - val_accuracy: 0.9529
Epoch 6/20
35/35 [==============================] - 97s 3s/step - loss: 0.2187 - accuracy: 0.9191 - val_loss: 0.2308 - val_accuracy: 0.9420
Epoch 7/20
35/35 [==============================] - 91s 3s/step - loss: 0.2367 - accuracy: 0.9118 - val_loss: 0.1578 - val_accuracy: 0.9565
Epoch 8/20
35/3

## Part 4 - Predicting face mask in real-time with OpenCV

In [14]:
# load the face classifier and initiate camera
face_recog=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source=cv2.VideoCapture(0)
# Define label and color dictionaries
labels_dic={0:'WITH MASK',1:'WITHOUT MASK'}
color_dic={0:(0,255,0),1:(0,0,255)}

In [15]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_recog.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,img_size) # resize the face image
        normalized=resized/255.0 # normalize the image
        img_exp=np.expand_dims(normalized,axis=0)
        result=cnn.predict(img_exp)  # make prediction
        label= np.argmax(result,axis=1)[0]

        cv2.rectangle(img,(x,y),(x+w,y+h),color_dic[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dic[label],-1)
        mask_label = "{}: {:.2f}%".format(labels_dic[label], round(np.max(result,axis=1)[0]*100,2))
        cv2.putText(img, mask_label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2) 
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

This model was motivated by the work of [The Perceptron](https://github.com/aieml/face-mask-detection-keras).
The image dataset was prepared by [Prajna Bhandary](https://github.com/prajnasb/observations/tree/master/experiements/data)